1. Read file
    - generated chunks results
    - Prepare dataframe for genearting new suggestion list
2. RERANK

3. Merge to original df
    - Get old dataframe
    - Merge with new suggestion list
4. Evaluate

In [1]:
import pandas as pd

# Do not truncate columns of DataFrame
pd.set_option('display.max_rows', None)
# Do not truncate column width
pd.set_option('display.max_colwidth', None)

import helpers_reranking as helpers

## 1 - Read file

In [2]:
#df = pd.read_csv('chunk_generation_2021-12-08_results.csv', sep='|')
# TODO: path + filename
#path = r'./Output_chunk/chunk_generation_2022-01-04_boyd_results.csv'
path = r'./Output_chunk/chunk_generation_2022-01-12_boyd_results.csv'
df = pd.read_csv(path, sep='|')

In [3]:
df.tail(10)
df.head(10)

,chunk,candidate,err_idx,num_chunk_build,prob
0,. Dodo ballt den Eismann,ballt,10,0,-123.696434
1,. Dodo bellt den Eismann,bellt,10,0,-123.541573
2,. Dodo beult den Eismann,beult,10,0,-124.366470
3,. Dodo bellst den Eismann,bellst,10,0,-124.366470
4,. Dodo hebelt den Eismann,hebelt,10,0,-123.764412
5,. Dodo beeilt den Eismann,beeilt,10,0,-123.298592
6,. Dodo pellt den Eismann,pellt,10,0,-216.808502
7,. Dodo ballte den Eismann,ballte,10,0,-124.251785
8,. Dodo Bild den Eismann,Bild,10,0,-121.537323
9,. Dodo gelt den Eismann,gelt,10,0,-123.968529


In [16]:
df.head(40)
len(df)

246010

## 2 - Rerank

In [3]:
# Insert loop here
new_df = helpers.rerank_suggs_by_prob(df, set_candidates=3)

In [11]:
new_df.tail(10)

,err_idx,num_chunk_build,new_suggestions
24591,210710,24591,"[nicht, Nicht, nichts, sieht, zieht, riecht, einsieht, näht, niest, niet]"
24592,210711,24592,"[dessen, vergessen, fassen, Wessen, wessen, verlesen, fesseln, Fesseln, fressen, vermessen]"
24593,210715,24593,"[eigentlich, endlich, Endlich, ängstlich, Ängstlich, ernstlich, sämtlich, ländlich, endlisch, hundlich]"
24594,210730,24594,"[Tür, Tor, Tore, Toren, Tenor, Tour, Torf, Thor, Tors, Tortes]"
24595,210732,24595,"[umarmte, Umarmte, umarmten, umarmt, unartigen, rumorte, zumachte, unartiges, bummerte, unartiger]"
24596,210743,24596,"[nicht, Nicht, nichts, sieht, zieht, riecht, einsieht, näht, niest, niet]"
24597,210749,24597,"[dass, aus, aß, pass, lass, fass, Ass, bass, hasst, nass]"
24598,210752,24598,"[nutzte, nützte, benutzten, nutzten, benutzte, putzte, benützte, benutztes, motzte, muckste]"
24599,210754,24599,"[Normen, Moment, Momenten, Nomen, Momente, Nomens, benommen, Moments, Mom, mopsen]"
24600,210760,24600,"[Lutschen, lutschen, Lutscher, lutschte, Löscher, lutsche, lutschten, Lutscht, Lutschphasen, lutscht]"


## 3 - Merge to original dataframe
- import old df
- prepare
- merge
- clean df from unnecessary columns; Goal: Look like normal analysis df, be able to evaluate

In [ ]:
#old_df=pd.read_pickle('data_error_token_language_tool_whole_texts_evaluation_lc_ALL_COLUMNS_OFFSETS_OTHERINDEX_20211208_TOMERGEWITH.pkl')
old_df=pd.read_pickle('data_error_types_boyd_evaluation_cs_preprocessed_reranking.pkl')

In [16]:
#old_df.tail(50)

In [21]:
# Change
old_df.rename(columns={'index':'err_idx'}, inplace=True)

In [22]:
old_df.head(2)
#old_df.shape[0]

,err_idx,original,corrected,filename,num_texts,length,start_position,end_position,ct,suggestions,rule_id,freq_ori,freq_cor,freq_tup,in_sugg,sugg_idx,idx0,idx_df_orig
0,10,belt,bellt,01-005-2-III-Eis.csv,0,4,46,50,10,"[Belt, Welt, bald, hält, Geld, Feld, Held, Zel...",GERMAN_SPELLER_RULE,91,138,91,True,9.0,False,0
1,17,kukt,kuckt,01-005-2-III-Eis.csv,0,4,80,84,17,"[rückt, zückt, bückt, guckt, kickt, kackt, gut...",GERMAN_SPELLER_RULE,73,152,73,False,NaN,False,1


In [23]:
merge_df=old_df.merge(new_df, how='left', on='err_idx')

In [26]:
merge_df.shape[0] # 24601
merge_df.head(2)

,err_idx,original,corrected,filename,num_texts,length,start_position,end_position,ct,suggestions,rule_id,freq_ori,freq_cor,freq_tup,in_sugg,sugg_idx,idx0,idx_df_orig,num_chunk_build,new_suggestions
0,10,belt,bellt,01-005-2-III-Eis.csv,0,4,46,50,10,"[Belt, Welt, bald, hält, Geld, Feld, Held, Zel...",GERMAN_SPELLER_RULE,91,138,91,True,9.0,False,0,0,"[Bild, beeilt, bellt, ballt, hebelt, gelt, bal..."
1,17,kukt,kuckt,01-005-2-III-Eis.csv,0,4,80,84,17,"[rückt, zückt, bückt, guckt, kickt, kackt, gut...",GERMAN_SPELLER_RULE,73,152,73,False,NaN,False,1,1,"[Guckt, guckt, Guckte, kickt, kickte, kickten,..."


In [27]:
# Do not truncate df value
pd.set_option('display.max_colwidth', None)

In [28]:
# Clean df from columns
#gr = ['err_idx','num_texts','length','start_position','end_position','ct','rule_id','in_sugg','sugg_idx','idx0','idx_df_orig','num_chunk_build']
gr = ['err_idx','num_texts','length','start_position','end_position','ct','idx_df_orig','num_chunk_build']
merge_df.drop(axis=1,inplace=True,labels=gr)

In [29]:
merge_df.tail(4)

,original,corrected,filename,suggestions,rule_id,freq_ori,freq_cor,freq_tup,in_sugg,sugg_idx,idx0,new_suggestions
24597,ass,aß,10-693-4-IV-Weg.csv,"[Ass, aß, als, aus, dass, ans, saß, Hass, Fass, nass, bass, lass, pass, Bass, Pass, aßt, iss, äse, äst, des]",GERMAN_SPELLER_RULE,3,7,3,True,1.0,False,"[dass, aus, aß, pass, lass, fass, Ass, bass, hasst, nass]"
24598,mutzte,nutzte,10-693-4-IV-Weg.csv,"[musste, nutzte, müsste, nützte, mutete, bützte, motzte, münzte, putzte, setzte, letzte, mussten, müsse, nutzt, Hütte, Künste, meiste, müssten, nutzten, wusste]",GERMAN_SPELLER_RULE,1,1,1,True,1.0,False,"[nutzte, nützte, benutzten, nutzten, benutzte, putzte, benützte, benutztes, motzte, muckste]"
24599,momen,Moment,10-693-4-IV-Weg.csv,"[mögen, Domen, mimen, zoomen, Nomen, boomen, Namen, kamen, hohen, oben, Polen, Roman, hören, roten, zogen, Boden, lösen, töten, Böhmen, Löwen]",GERMAN_SPELLER_RULE,1,4,1,False,NaN,False,"[Normen, Moment, Momenten, Nomen, Momente, Nomens, benommen, Moments, Mom, mopsen]"
24600,Lutscha,Lutscher,10-693-4-IV-Weg.csv,"[Lutsch, Luigi, Lutsche, Lunge, Lutscht, Putsch, Kutsche, Rutscht, Ludger, Lusaka, Putsche, Luchs, Rutsche, Luchse, Rutsch, Latsche, Blutschau, Datscha, Dougga, Hodscha]",GERMAN_SPELLER_RULE,8,56,8,False,NaN,False,"[Lutschen, lutschen, Lutscher, lutschte, Löscher, lutsche, lutschten, Lutscht, Lutschphasen, lutscht]"


In [24]:
#merge_df.tail(100)

In [25]:
merge_df_new = merge_df.drop(axis=1,labels='suggestions')

In [26]:
merge_df_new.rename(columns={'new_suggestions':'suggestions'}, inplace=True)

In [27]:
merge_df_new.head(20)

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[Bild, beeilt, bellt, ballt, hebelt, gelt, ballte, beult, bellst, pellt]"
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,"[Guckt, guckt, Guckte, kickt, kickte, kickten, Guckst, guckst, kuckt, kickst]"
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[dann, Dann, den, dank, darin, Darin, dar, dran, Dan, Dean]"
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,"[Gericht, gekriegt, gekratzt, gekracht, gegrillt, verkriecht, gekreischt, Gekriech, gekrächzt, verkriechst]"
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,"[legen, Legen, legten, lenken, Lenken, lecken, Lecken, leckten, leckend, blecken]"
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,"[fällt, Fällt, fehlt, Fehlt, Feld, verhält, feilt, fallt, Fallt, fackelt]"
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,"[wollte, Wollte, wollten, Wollten, wolle, wohnte, Volte, wölbte, wolltet, wegholte]"
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,"[legt, Legt, lenkt, leckte, lenk, leckt, Leckt, legst, leckten, lenkst]"
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,"[vom, Vom, Form, vorn, Vorn, vorm, Vorm, flop, formt, fromm]"
9,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[dann, Dann, dank, den, darin, Darin, dar, dran, Dan, Dean]"


## 4 - Evaluate

In [36]:
# EVALUIEREN
# Prep
merge_df_new.reset_index(drop=True, inplace=True)
# Not needed because there are suggestions for every type
#merge_df_new.suggestions.fillna(value='',inplace=True)

In [37]:
import sys
sys.path.insert(0, '../../Lisa')
import litkey_2
import helpers_litkey_2 as helpers

In [38]:
type(merge_df_new.suggestions[1][5])

str

In [39]:
token_eval_lc = litkey_2.evaluate(merge_df_new)

  0%|          | 0/24601 [00:00<?, ?it/s]

In [40]:
token_eval_cs = litkey_2.evaluate(merge_df_new, lower_case=False)

  0%|          | 0/24601 [00:00<?, ?it/s]

In [42]:
token_eval_cs.head(50)

,original,corrected,filename,suggestions_,rule_id,freq_ori,freq_cor,freq_tup,in_sugg_,sugg_idx_,idx0_,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,"[Belt, Welt, bald, hält, Geld, Feld, Held, Zelt, bebt, bellt, perlt, pell, best, meld, peilt, pellt, pult, zelt, bei, seit]",GERMAN_SPELLER_RULE,91,138,91,True,9.0,False,"[Bild, beeilt, bellt, ballt, hebelt, gelt, ballte, beult, bellst, pellt]",True,2.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,"[rückt, zückt, bückt, guckt, kickt, kackt, gut, rückte, deckt, fügt, Lücke, lockt, weckt, packt, Mücke, kund, kürt, lügt, hackt, hockt]",GERMAN_SPELLER_RULE,73,152,73,False,NaN,False,"[Guckt, guckt, Guckte, kickt, kickte, kickten, Guckst, guckst, kuckt, kickst]",True,8.0,False
2,dan,dann,01-005-2-III-Eis.csv,"[dann, Dan, den, an, das, man, da, dar, van, San, tun, Jan, dank, tat, dran, ran, tank, Fan, Idan, Pan]",GERMAN_SPELLER_RULE,627,651,621,True,0.0,True,"[dann, Dann, den, dank, darin, Darin, dar, dran, Dan, Dean]",True,0.0,True
3,gekricht,gekriegt,01-005-2-III-Eis.csv,"[gebricht, gegeigt, gekickt, gekracht, gebracht, geprägt, gezeigt, Gericht, Bericht, gefragt, gerecht, gekocht, geneigt, gereicht, Rücksicht, gerückt, geklagt, geeicht, gegrillt, geknickt]",GERMAN_SPELLER_RULE,2,15,2,False,NaN,False,"[Gericht, gekriegt, gekratzt, gekracht, gegrillt, verkriecht, gekreischt, Gekriech, gekrächzt, verkriechst]",True,1.0,False
4,leken,lecken,01-005-2-III-Eis.csv,"[legen, locken, lägen, Säcken, lecken, gegen, wegen, leben, lagen, lesen, Becken, decken, dicken, legten, lenken, locker, regen, rücken, Lücken, Nacken]",GERMAN_SPELLER_RULE,14,17,14,True,4.0,False,"[legen, Legen, legten, lenken, Lenken, lecken, Lecken, leckten, leckend, blecken]",True,5.0,False
5,felt,fällt,01-005-2-III-Eis.csv,"[Welt, fest, fällt, hält, Geld, fehlt, Feld, Held, fett, Zelt, fegt, Belt, feilt, feit, fielt, meld, pfählt, zelt, seit]",GERMAN_SPELLER_RULE,93,198,90,True,2.0,False,"[fällt, Fällt, fehlt, Fehlt, Feld, verhält, feilt, fallt, Fallt, fackelt]",True,0.0,True
6,wolte,wollte,01-005-2-III-Eis.csv,"[wollte, holte, volle, wohnte, wolle, wilde, Nolte, Nolde, Bolte, Dolde, holde, wölbte, Golde, Solde, Volte, johlte, nölte, vote, walte, wogte]",GERMAN_SPELLER_RULE,173,201,173,True,0.0,True,"[wollte, Wollte, wollten, Wollten, wolle, wohnte, Volte, wölbte, wolltet, wegholte]",True,0.0,True
7,lekt,leckt,01-005-2-III-Eis.csv,"[legt, lockt, leckt, bäckt, lägt, lebt, legte, deckt, lädt, rückt, lenkt, lockte, weckt, Bäcker, hegt, lege, leid, läge, packt, regt]",GERMAN_SPELLER_RULE,20,42,19,True,2.0,False,"[legt, Legt, lenkt, leckte, lenk, leckt, Leckt, legst, leckten, lenkst]",True,5.0,False
8,fom,vom,01-005-2-III-Eis.csv,"[FOM, vom, von, vor, Rom, Dom, vorm, zoom, Boom, Tom, Zoom, im, dem, für, am, um, zum, so, ihm, kam]",GERMAN_SPELLER_RULE,13,16,13,True,1.0,False,"[vom, Vom, Form, vorn, Vorn, vorm, Vorm, flop, formt, fromm]",True,0.0,True
9,dan,dann,01-005-2-III-Eis.csv,"[dann, Dan, den, an, das, man, da, dar, van, San, tun, Jan, dank, tat, dran, ran, tank, Fan, Idan, Pan]",GERMAN_SPELLER_RULE,627,651,621,True,0.0,True,"[dann, Dann, dank, den, darin, Darin, dar, dran, Dan, Dean]",True,0.0,True


In [34]:
# To pickle
#token_eval_lc.to_pickle('./pickles/reranking_data_error_token_language_tool_whole_texts_evaluation_lc.pkl')
#token_eval_cs.to_pickle('./pickles/reranking_data_error_token_language_tool_whole_texts_evaluation_cs.pkl')

In [35]:
# TODO: Set path dynamically
token_eval_lc.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_lc_NEW.pkl')
token_eval_cs.to_pickle('./pickles/reranking_data_error_token_boyd_evaluation_cs_NEW.pkl')

In [36]:
#token_eval_lc = pd.read_pickle('./pickles/reranking_data_error_token_boyd_evaluation_lc.pkl')
#token_eval_cs = pd.read_pickle('./pickles/reranking_data_error_token_boyd_evaluation_cs.pkl')

In [37]:
token_eval_cs.head(5)

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[Bild, beeilt, bellt, ballt, hebelt, gelt, ballte, beult, bellst, pellt]",True,2.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,"[Guckt, guckt, Guckte, kickt, kickte, kickten, Guckst, guckst, kuckt, kickst]",True,8.0,False
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[dann, Dann, den, dank, darin, Darin, dar, dran, Dan, Dean]",True,0.0,True
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,"[Gericht, gekriegt, gekratzt, gekracht, gegrillt, verkriecht, gekreischt, Gekriech, gekrächzt, verkriechst]",True,1.0,False
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,"[legen, Legen, legten, lenken, Lenken, lecken, Lecken, leckten, leckend, blecken]",True,5.0,False
